In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("../crsp_daily_cut.csv", index_col=0)
df

,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr
109545,10026,2023-01-05,11,3,2052,149.60,-0.010713,60472.0,19229.0
109546,10026,2023-01-06,11,3,2052,152.64,0.020321,57034.0,19229.0
109547,10026,2023-01-09,11,3,2052,149.61,-0.019851,63214.0,19229.0
109548,10026,2023-01-10,11,3,2052,149.87,0.001738,112759.0,19229.0
109549,10026,2023-01-11,11,3,2052,152.39,0.016815,57350.0,19229.0
...,...,...,...,...,...,...,...,...,...
105258375,93436,2023-12-22,11,3,9999,252.54,-0.007701,93148500.0,3178920.0
105258376,93436,2023-12-26,11,3,9999,256.61,0.016116,86700700.0,3178920.0
105258377,93436,2023-12-27,11,3,9999,261.44,0.018822,106251000.0,3178920.0
105258378,93436,2023-12-28,11,3,9999,253.18,-0.031594,113251000.0,3178920.0
